# Chapter 9: A Pythonic object

In [85]:
# A 2d vector object
import math
from array import array

class Vector:
    # Typecode for turing into bytes
    typecode = 'd'

    # __slots__ so attributes are stored as tuples instead of a dict
    # which saves memory (only an issue if instanciating millions of objects)
    __slots__ = ('__x', '__y')
    
    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)
        
    def __iter__(self):
        return (i for i in (self.x, self.y))
        
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(*self)
    
    def __bool__(self):
        return bool(abs(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(array(self.typecode, self)))
    
    #def __format__(self, fmt_spec=''):
    #    components = (format(coord, fmt_spec) for coord in self)
    #    return '({}, {})'.format(*components)
    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle)
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(item, fmt_spec) for item in coords)
        return outer_fmt.format(*components)
        
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)
    
    @property
    def angle(self):
        """ venctor angle (in radians) """
        return math.atan2(*self)
    
    # Make it hashable so it can be used in sets and as dict keys
    # Must make immutable first (attributes as read-only)
    @property
    def x(self):
        return self.__x
    
    @property
    def y(self):
        return self.__y
    
    def __hash__(self):
        """
        The __hash__ docs recomend using the polar operator ^ (XOR) to mix
        hashes of components
        """
        return hash(self.x) ^ hash(self.y)
    

In [71]:
v = Vector(1,2)
print('coords:',f'{v:.4f}')
print('polar:', f'{v:.5p}')

coords: (1.0000, 2.0000)
polar: <2.2361, 0.46365>


In [82]:
# private attributes
print(v.x)
v.x = 5

1.0


AttributeError: can't set attribute

In [79]:
# Hashable
v = Vector(1,2)
v1 = Vector(1,2)
v2 = Vector(1.4, 2.5)
v_names = 'v1 v2 v2'.split()
v_list = [v, v1, v2]
print(v == v1)
print(hash(v), hash(v1))
print(set(v_list))
print(dict(zip(v_names, v_list)))

True
3 3
{Vector(1.4, 2.5), Vector(1.0, 2.0)}
{'v1': Vector(1.0, 2.0), 'v2': Vector(1.4, 2.5)}


In [92]:
# Class attributes
# are passed on to instances by default, and can be overriden.
# The more pythonic way is to create a subclass than only alters 
# the desired attribures
class ShortVector(Vector):
    typecode = 'f'
    
v1 = Vector(1/11, 1/27)
v2 = ShortVector(1/7, 1/23)

print(len(bytes(v1)))
print(len(bytes(v2)))


17
9
